<a href="https://colab.research.google.com/github/NimishK26/Data-Management-WBS/blob/main/Data_Generation_DM_WBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests faker pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.0 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import random
import string
import re
import datetime
from faker import Faker
# Initialize Faker for UK locale
fake = Faker("en_GB")
# List of 10 major cities in the UK (Most customers will be from
these)
big_cities = ["London", "Manchester", "Birmingham", "Glasgow",
"Liverpool",
"Leeds", "Edinburgh", "Bristol", "Sheffield",
"Nottingham"]
# Expanding the list of first names for Male, Female, and Other
categories
male_names = [
"James", "John", "Robert", "Michael", "William", "David",
"Richard", "Joseph", "Thomas", "Charles",
"Daniel", "Matthew", "Luke", "Edward", "Harry", "George", "Jack",
"Oliver", "Henry", "Samuel",
"Jake", "Nathan", "Lewis", "Ryan", "Oscar", "Alex", "Ethan",
"Liam", "Benjamin", "Joshua",
"Noah", "Charlie", "Adam", "Connor", "Zachary", "Harrison",
"Toby", "Callum", "Jayden", "Arthur"
]
female_names = [
"Mary", "Patricia", "Jennifer", "Linda", "Elizabeth", "Barbara",
"Susan", "Jessica", "Sarah", "Karen",
"Emily", "Hannah", "Charlotte", "Sophie", "Olivia", "Isabella",
"Amelia", "Megan", "Abigail", "Emma",
"Lucy", "Katie", "Ellie", "Lauren", "Rebecca", "Holly", "Jasmine",
"Eleanor", "Phoebe", "Freya",
"Madison", "Alice", "Isla", "Anna", "Mia", "Amber", "Daisy",
"Harriet", "Erin", "Victoria"
]
# Other gender-neutral names (Only 4% of dataset)
other_names = [
"Taylor", "Jordan", "Morgan", "Casey", "Jamie", "Alexis", "Robin",
"Skyler", "Avery", "Riley",
"Finley", "Sasha", "Dakota", "Phoenix", "Quinn", "Eden", "River",
"Rowan", "Harper", "Indigo"
]
# Merging all names into one dictionary with category labels
all_names = {
"Male": male_names,
"Female": female_names,
"Other": other_names
}
# Adjusted gender distribution: 53% Female, 43% Male, 4% Other
name_distribution = ["Female"] * 53 + ["Male"] * 43 + ["Other"] * 4
# Function to generate a name and assign gender accordingly
def generate_name_and_gender():
gender = random.choice(name_distribution) # Assign gender based
on adjusted distribution
first_name = random.choice(all_names[gender]) # Select a name
from the respective category
return first_name, gender
# Function to generate a unique customer ID (CUST followed by 5
digits)
def generate_customer_id():
return f"CUST{random.randint(10000, 99999)}"
# Function to fetch UK addresses, prioritizing big cities
def get_address_details():
if random.random() < 0.75: # 75% chance to choose a big city
city = random.choice(big_cities)
return {
"postcode": f"{city[:2].upper()}{random.randint(1, 9)}
{random.choice('ABCDEFGHJKLMNPRSTUVWXYZ')}{random.randint(1,
9)}{random.choice('ABCDEFGHJKLMNPRSTUVWXYZ')}",
"country": "United Kingdom",
"city": city,
"street": fake.street_name()
}
else:
response =
requests.get("https://api.postcodes.io/random/postcodes")
if response.status_code == 200:
data = response.json()["result"]
return {
"postcode": data["postcode"],
"country": "United Kingdom",
"city": data["admin_district"],
"street": fake.street_name()
}
return None
# Function to generate a random UK mobile number
def generate_random_uk_mobile_number():
return f"07{random.randint(100000000, 999999999)}"
# Function to validate UK phone number format (Must start with 07 and
be 11 digits long)
def validate_uk_mobile_number(number):
return re.match(r"^07\d{9}$", number) is not None # Checks for
correct format
# Function to validate email format
def validate_email(email):
email_regex = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z.]+$"
allowed_tlds = (".com", ".co.uk", ".org", ".net") # Only valid
TLDs
if not re.match(email_regex, email) or not
email.endswith(allowed_tlds):
return None
return email
# Function to generate a realistic date of birth with most in 1985-
2005 range
def generate_dob():
if random.random() < 0.7: # 70% chance for DOB between 1985 and
2005
year = random.randint(1985, 2005)
else: # 30% chance for DOB between 1965 and 1984 or 2006-2010
year = random.choice(range(1965, 1985)) if random.random() <
0.5 else random.choice(range(2006, 2010))
month = random.randint(1, 12)
day = random.randint(1, 28) # Keep within valid range for all months
return datetime.date(year, month, day).strftime("%Y-%m-%d")
# Generate customer data with at least 700 unique records
num_rows = 700
customer_data = set()
while len(customer_data) < num_rows:
customer_id = generate_customer_id()
first_name, gender = generate_name_and_gender() # Generate name
and gender
last_name = fake.last_name()
email =
validate_email(f"{first_name.lower()}.{last_name.lower()}@{random.choi
ce(['gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com'])}")
if not email:
continue # Skip invalid emails
phone = None
while not phone:
random_phone = generate_random_uk_mobile_number()
if validate_uk_mobile_number(random_phone): # Check phone
format
phone = random_phone
address_details = get_address_details()
if not address_details:
continue # Skip if address details are not found
dob = generate_dob() # Generate Date of Birth
customer_data.add((
customer_id, gender, first_name, last_name, dob, email, phone,
address_details["street"], address_details["city"],
address_details["postcode"], address_details["country"]
                    ))
# Convert set to DataFrame
customer_df = pd.DataFrame(list(customer_data), columns=[
"CustomerID", "Gender", "First Name", "Last Name", "Date of
Birth", "Email", "Phone",
"Street", "City", "Postcode", "Country"
])
# Save the dataset to CSV
customer_df.to_csv("customer_uk_database.csv", index=False)
print("CSV file 'customer_uk_database.csv' generated successfully!")